# New Native Contacts Analysis

Functions to load hydrogen bond analysis results and convert to pandas DataFrames for analysis.

In [1]:
import numpy as np
import pandas as pd
import pickle
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import List, Dict, Tuple, Optional

In [2]:
def load_hbond_data_to_dataframe(file_prefix, results_dir="results/", show_timing=False):
    """
    Load hydrogen bond analysis results and convert to pandas DataFrame.
    
    Args:
        file_prefix: e.g. "Kmarx_Pab1.run.0"
        results_dir: directory containing the results files
        show_timing: whether to print timing information
    
    Returns:
        pandas.DataFrame with columns:
        - frame: trajectory frame number
        - residue_i: first residue index
        - residue_j: second residue index  
        - hbond_energy: hydrogen bond energy between residues
        - run_id: extracted from filename for identification
    """
    start_time = time.time()
    
    # Construct file paths
    npy_file = Path(results_dir) / f"{file_prefix}_hbond_energy_maps.npy"
    pkl_file = Path(results_dir) / f"{file_prefix}_hbond_results.pkl"
    
    # Check if files exist
    if not npy_file.exists():
        raise FileNotFoundError(f"Energy maps file not found: {npy_file}")
    if not pkl_file.exists():
        raise FileNotFoundError(f"Results file not found: {pkl_file}")
    
    # Load data
    load_start = time.time()
    energy_maps = np.load(npy_file)  # Shape: (n_frames, n_residues, n_residues)
    
    with open(pkl_file, 'rb') as f:
        metadata = pickle.load(f)
    load_time = time.time() - load_start
    
    n_frames, n_residues, _ = energy_maps.shape
    
    # Extract run_id from file_prefix
    run_id = file_prefix.split('.')[-1] if '.' in file_prefix else file_prefix
    
    # Convert to long format DataFrame
    process_start = time.time()
    data_rows = []
    
    for frame in range(n_frames):
        for i in range(n_residues):
            for j in range(i+1, n_residues):  # Only upper triangle to avoid duplicates
                energy = energy_maps[frame, i, j]
                if energy > 1e-6:  # Only include meaningful interactions
                    data_rows.append({
                        'frame': frame,
                        'residue_i': i,
                        'residue_j': j,
                        'hbond_energy': energy,
                        'run_id': run_id
                    })
    
    df = pd.DataFrame(data_rows)
    process_time = time.time() - process_start
    
    # Add metadata as attributes
    df.attrs = {
        'n_donors': len(metadata['donors']),
        'n_acceptors': len(metadata['acceptors']),
        'n_residues': metadata['n_residues'],
        'n_frames': metadata['n_frames'],
        'file_prefix': file_prefix
    }
    
    total_time = time.time() - start_time
    
    if show_timing:
        print(f"\nTiming for {file_prefix}:")
        print(f"  File loading: {load_time:.3f}s")
        print(f"  Data processing: {process_time:.3f}s")
        print(f"  Total time: {total_time:.3f}s")
        print(f"  Interactions found: {len(df)}")
        print(f"  Processing rate: {len(df)/total_time:.0f} interactions/sec")
    
    return df

In [3]:
def save_dataframe_to_csv(df, filepath, include_metadata=True):
    """
    Save DataFrame to CSV with optional metadata preservation.
    
    Args:
        df: pandas DataFrame to save
        filepath: path where to save the CSV
        include_metadata: whether to save metadata as a separate file
    """
    filepath = Path(filepath)
    filepath.parent.mkdir(parents=True, exist_ok=True)
    
    # Save main DataFrame
    df.to_csv(filepath, index=False)
    
    # Save metadata if available and requested
    if include_metadata and hasattr(df, 'attrs') and df.attrs:
        metadata_file = filepath.with_suffix('.meta.json')
        import json
        with open(metadata_file, 'w') as f:
            json.dump(df.attrs, f, indent=2)
    
    print(f"Saved DataFrame to {filepath}")
    if include_metadata:
        print(f"Saved metadata to {filepath.with_suffix('.meta.json')}")

def load_dataframe_from_csv(filepath, load_metadata=True):
    """
    Load DataFrame from CSV with optional metadata restoration.
    
    Args:
        filepath: path to the CSV file
        load_metadata: whether to load metadata from separate file
    
    Returns:
        pandas DataFrame with restored metadata if available
    """
    filepath = Path(filepath)
    
    # Load main DataFrame
    df = pd.read_csv(filepath)
    
    # Load metadata if available and requested
    if load_metadata:
        metadata_file = filepath.with_suffix('.meta.json')
        if metadata_file.exists():
            import json
            with open(metadata_file, 'r') as f:
                df.attrs = json.load(f)
    
    print(f"Loaded DataFrame from {filepath}")
    return df

## Native Contacts Analyzer Class

Comprehensive system for analyzing native hydrogen bond contacts over time.

In [4]:
class NativeContactsAnalyzer:
    """
    Comprehensive analysis system for native hydrogen bond contacts.
    
    - Loading runs individually and storing in indexed list
    - CSV save/load functionality 
    - Native contacts identification from frame 0
    - Time series analysis of native contacts preservation
    - Plotting capabilities
    
    Note: From June 2024, `energy_fraction` is defined as:
      (sum of energies of native contacts present in current frame) /
      (sum of all hbond energies present in current frame above threshold)
    """
    
    def __init__(self, results_dir="results/", csv_dir="results/csv_data/"):
        self.results_dir = Path(results_dir)
        self.csv_dir = Path(csv_dir)
        self.csv_dir.mkdir(parents=True, exist_ok=True)
        
        self.runs_data = []  # List indexed by run number
        self.native_contacts = None  # Native contacts from frame 0
        self.native_contact_energies = None  # Native contact energies from frame 0
    
    def load_single_run(self, run_number: int, file_pattern="Kmarx_Pab1.run", 
                       save_csv=True, show_timing=False):
        """
        Load a single run and optionally save as CSV.
        
        Args:
            run_number: run number to load
            file_pattern: pattern for finding files
            save_csv: whether to save DataFrame as CSV
            show_timing: whether to show timing information
        """
        file_prefix = f"{file_pattern}.{run_number}"
        
        try:
            # Load the run
            df = load_hbond_data_to_dataframe(file_prefix, self.results_dir, show_timing)
            
            # Ensure runs_data list is large enough
            while len(self.runs_data) <= run_number:
                self.runs_data.append(None)
            
            # Store in list at correct index
            self.runs_data[run_number] = df
            
            # Save as CSV if requested
            if save_csv:
                csv_path = self.csv_dir / f"run_{run_number:03d}.csv"
                save_dataframe_to_csv(df, csv_path)
            
            print(f"Successfully loaded run {run_number} with {len(df)} interactions")
            return df
            
        except FileNotFoundError as e:
            print(f"Could not load run {run_number}: {e}")
            return None
    
    def load_all_runs(self, file_pattern="Kmarx_Pab1.run", max_runs=None, 
                     save_csv=True, show_timing=False):
        """
        Load all available runs.
        
        Args:
            file_pattern: pattern for finding files
            max_runs: maximum number of runs to load (None for all)
            save_csv: whether to save DataFrames as CSV
            show_timing: whether to show timing information
        """
        # Find all available runs
        npy_files = list(self.results_dir.glob(f"{file_pattern}*_hbond_energy_maps.npy"))
        
        # Extract run numbers
        run_numbers = []
        for f in npy_files:
            prefix = f.stem.replace("_hbond_energy_maps", "")
            try:
                run_num = int(prefix.split('.')[-1])
                run_numbers.append(run_num)
            except ValueError:
                continue
        
        run_numbers.sort()
        if max_runs:
            run_numbers = run_numbers[:max_runs]
            
        print(f"Found {len(run_numbers)} runs to load: {run_numbers}")
        
        # Load each run
        successful_loads = 0
        for run_num in run_numbers:
            df = self.load_single_run(run_num, file_pattern, save_csv, show_timing)
            if df is not None:
                successful_loads += 1
        
        print(f"Successfully loaded {successful_loads} out of {len(run_numbers)} runs")
        return successful_loads
    
    def identify_native_contacts(self, run_number=0, energy_threshold=1e-6):
        """
        Identify native contacts from frame 0 of a reference run.
        
        Args:
            run_number: which run to use as reference (default: 0)
            energy_threshold: minimum energy to consider a contact
        """
        if run_number >= len(self.runs_data) or self.runs_data[run_number] is None:
            raise ValueError(f"Run {run_number} not loaded")
        
        df = self.runs_data[run_number]
        
        # Get frame 0 data
        frame_0 = df[df['frame'] == 0].copy()
        
        # Native contacts are residue pairs with energy > threshold at frame 0
        native_pairs = frame_0[frame_0['hbond_energy'] > energy_threshold]
        
        # Store native contact pairs (set of tuples)
        self.native_contacts = set()
        for _, row in native_pairs.iterrows():
            pair = (min(row['residue_i'], row['residue_j']), 
                   max(row['residue_i'], row['residue_j']))
            self.native_contacts.add(pair)
        
        # Store native contact energies (dict of pair -> energy)
        self.native_contact_energies = {}
        for _, row in native_pairs.iterrows():
            pair = (min(row['residue_i'], row['residue_j']), 
                   max(row['residue_i'], row['residue_j']))
            self.native_contact_energies[pair] = row['hbond_energy']
        
        print(f"Identified {len(self.native_contacts)} native contacts from run {run_number}, frame 0")
        print(f"Total native contact energy: {sum(self.native_contact_energies.values()):.3f}")
        
        return self.native_contacts, self.native_contact_energies
    
    def calculate_native_contacts_timeseries(self, run_number: int, energy_threshold=1e-6):
        """
        Calculate native contacts preservation over time for a single run.

        Args:
            run_number: run to analyze
            energy_threshold: minimum energy to consider a contact present
            
        Returns:
            DataFrame with columns: frame, count_fraction, energy_fraction

        energy_fraction = (sum of energies for native contacts in current frame) /
                          (sum of all hbond energies in current frame > threshold)
        """
        if self.native_contacts is None:
            raise ValueError("Native contacts not identified. Run identify_native_contacts() first.")
        
        if run_number >= len(self.runs_data) or self.runs_data[run_number] is None:
            raise ValueError(f"Run {run_number} not loaded")
        
        df = self.runs_data[run_number]
        
        frames = sorted(df['frame'].unique())
        results = []
        
        total_native_contacts = len(self.native_contacts)
        
        for frame in frames:
            frame_data = df[df['frame'] == frame]
            
            # Get total energy in this frame above threshold (all contacts)
            current_total_energy = frame_data.loc[
                frame_data['hbond_energy'] > energy_threshold, 'hbond_energy'
            ].sum()
            
            present_native_contacts = 0
            current_native_energy = 0.0

            # Check each native contact
            for pair in self.native_contacts:
                i, j = pair
                # Look for this pair in current frame
                contact_rows = frame_data[
                    ((frame_data['residue_i'] == i) & (frame_data['residue_j'] == j)) |
                    ((frame_data['residue_i'] == j) & (frame_data['residue_j'] == i))
                ]
                if not contact_rows.empty:
                    max_energy = contact_rows['hbond_energy'].max()
                    if max_energy > energy_threshold:
                        present_native_contacts += 1
                        current_native_energy += max_energy

            # Calculate fractions
            count_fraction = (
                present_native_contacts / total_native_contacts
                if total_native_contacts > 0 else 0
            )
            energy_fraction = (
                current_native_energy / current_total_energy
                if current_total_energy > 0 else 0
            )
            
            results.append({
                'frame': frame,
                'count_fraction': count_fraction,
                'energy_fraction': energy_fraction,
                'run_id': run_number
            })
        
        return pd.DataFrame(results)
    
    def calculate_all_native_contacts_timeseries(self, energy_threshold=1e-6):
        """
        Calculate native contacts preservation for all loaded runs.
        
        Returns:
            DataFrame with all runs' timeseries data
        """
        if self.native_contacts is None:
            raise ValueError("Native contacts not identified. Run identify_native_contacts() first.")
        
        all_results = []
        
        for run_num, df in enumerate(self.runs_data):
            if df is not None:
                print(f"Processing run {run_num}...")
                run_results = self.calculate_native_contacts_timeseries(run_num, energy_threshold)
                all_results.append(run_results)
        
        if not all_results:
            raise ValueError("No runs loaded")
        
        combined_df = pd.concat(all_results, ignore_index=True)
        return combined_df
    
    def plot_native_contacts_timeseries(self, timeseries_df=None, save_path=None, 
                                       figsize=(12, 8), show_individual_runs=True):
        """
        Plot native contacts preservation over time.
        
        Args:
            timeseries_df: DataFrame with timeseries data (if None, calculates it)
            save_path: path to save figure. You can use '{run_id}' as a placeholder in the path.
            figsize: figure size
            show_individual_runs: whether to show individual run traces
        """
        if timeseries_df is None:
            timeseries_df = self.calculate_all_native_contacts_timeseries()
        
        run_ids = timeseries_df['run_id'].unique()
        is_single_run = len(run_ids) == 1

        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize, sharex=True)
        
        # Plot 1: Count-based native contacts
        if show_individual_runs:
            # Show individual runs
            for run_id in run_ids:
                run_data = timeseries_df[timeseries_df['run_id'] == run_id]
                ax1.plot(run_data['frame'], run_data['count_fraction'] * 100, 
                        alpha=0.3, linewidth=0.8, color='blue')
        
        # Show average
        avg_count = timeseries_df.groupby('frame')['count_fraction'].mean()
        std_count = timeseries_df.groupby('frame')['count_fraction'].std()
        frames = avg_count.index
        
        ax1.plot(frames, avg_count * 100, 'b-', linewidth=2, label='Average')
        ax1.fill_between(frames, (avg_count - std_count) * 100, (avg_count + std_count) * 100, 
                        alpha=0.2, color='blue')
        
        ax1.set_ylabel('% Native Contacts (Count)', fontsize=12)
        ax1.set_title('Native Hydrogen Bond Contacts Over Time', fontsize=14)
        ax1.grid(True, alpha=0.3)
        ax1.legend()
        
        # Plot 2: Energy-based native contacts
        if show_individual_runs:
            # Show individual runs
            for run_id in run_ids:
                run_data = timeseries_df[timeseries_df['run_id'] == run_id]
                ax2.plot(run_data['frame'], run_data['energy_fraction'] * 100, 
                        alpha=0.3, linewidth=0.8, color='red')
        
        # Show average
        avg_energy = timeseries_df.groupby('frame')['energy_fraction'].mean()
        std_energy = timeseries_df.groupby('frame')['energy_fraction'].std()
        
        ax2.plot(frames, avg_energy * 100, 'r-', linewidth=2, label='Average')
        ax2.fill_between(frames, (avg_energy - std_energy) * 100, (avg_energy + std_energy) * 100, 
                        alpha=0.2, color='red')
        
        ax2.set_xlabel('Frame', fontsize=12)
        ax2.set_ylabel('% Native Contacts (Energy)', fontsize=12)
        ax2.grid(True, alpha=0.3)
        ax2.legend()
        
        plt.tight_layout()
        
        # ----- Save logic with run id -----
        if save_path:
            save_path = str(save_path)
            if '{run_id}' in save_path:
                # Save separate figure for each run if plotting single run,
                # or 'all' if plotting the combined average
                if is_single_run:
                    current_run_id = run_ids[0]
                else:
                    current_run_id = 'all'
                actual_save_path = save_path.replace('{run_id}', str(current_run_id))
            else:
                # Append run id or 'all' before file extension if not present
                p = Path(save_path)
                if is_single_run:
                    actual_save_path = str(p.with_name(f"{p.stem}_run{run_ids[0]}{p.suffix}"))
                else:
                    actual_save_path = str(p.with_name(f"{p.stem}_all{p.suffix}"))
            plt.savefig(actual_save_path, dpi=300, bbox_inches='tight')
            print(f"Saved plot to {actual_save_path}")
        
        plt.show()
        
        return fig

## Usage Examples

In [ ]:
# Create analyzer instance
analyzer = NativeContactsAnalyzer()

# Load all available runs (this will save CSVs automatically)
analyzer.load_all_runs(max_runs=48)  # Limit to first 5 runs for testing

# Identify native contacts from run 0, frame 0
analyzer.identify_native_contacts(run_number=0)

# Calculate time series for all runs
timeseries_data = analyzer.calculate_all_native_contacts_timeseries()

# Plot the results
analyzer.plot_native_contacts_timeseries(timeseries_data, 
                                        save_path="results/native_contacts_timeseries.png")

Found 48 runs to load: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
Saved DataFrame to results/csv_data/run_000.csv
Saved metadata to results/csv_data/run_000.meta.json
Successfully loaded run 0 with 81640 interactions
Saved DataFrame to results/csv_data/run_001.csv
Saved metadata to results/csv_data/run_001.meta.json
Successfully loaded run 1 with 86314 interactions
Saved DataFrame to results/csv_data/run_002.csv
Saved metadata to results/csv_data/run_002.meta.json
Successfully loaded run 2 with 82438 interactions
Saved DataFrame to results/csv_data/run_003.csv
Saved metadata to results/csv_data/run_003.meta.json
Successfully loaded run 3 with 82189 interactions
Saved DataFrame to results/csv_data/run_004.csv
Saved metadata to results/csv_data/run_004.meta.json
Successfully loaded run 4 with 80426 interactions
Saved DataFrame to results/csv_data/run

In [6]:
# Alternative: Load individual runs
# analyzer = NativeContactsAnalyzer()
# analyzer.load_single_run(0, show_timing=True)
# analyzer.load_single_run(1, show_timing=True)
# analyzer.load_single_run(2, show_timing=True)

In [7]:
# Load from saved CSV files
# analyzer = NativeContactsAnalyzer()
# analyzer.load_run_from_csv(0)
# analyzer.load_run_from_csv(1)

In [8]:
# Get summary statistics
# summary = analyzer.get_summary_statistics(timeseries_data)
# print("Summary Statistics:")
# for key, value in summary.items():
#     print(f"{key}: {value}")